In [13]:
import pandas as pd 
import cv2
import numpy as np
from sklearn.model_selection import train_test_split


In [14]:
# Read data from file 'filename.csv' 
# (in the same directory that your python process is based)
# Control delimiters, rows, column names with read_csv (see later) 
label = pd.read_csv('devset_images_gt.csv') 
# Preview the first 5 lines of the loaded data 
label_array = label.values


In [17]:
import os
import numpy as np
from PIL import Image
import tensorflow as tf

# Đường dẫn tới folder chứa ảnh
folder_path = "C:/Users/minhd/FPTU lab/DPL302m/Kaggle/2024-sum-dpl-302-m/devset_images/devset_images"

# Hàm để đọc và xử lý từng ảnh
def load_and_preprocess_image(image_path, target_size=(256, 256)):
    image = Image.open(image_path)
    image = image.resize(target_size)
    image = np.array(image)
    if image.shape[-1] == 4:  # Nếu ảnh có kênh alpha (RGBA), chuyển về RGB
        image = image[:, :, :3]
    image = image / 255.0  # Chuẩn hóa ảnh về khoảng [0, 1]
    return image

# Đọc tất cả các ảnh trong folder
images = []
Y_train = []

# List of allowed file extensions
allowed_extensions = ['.jpg', '.png', '.gif']

# Giả sử label_array là một mảng các tên file và nhãn tương ứng
# Cập nhật lại danh sách các file ảnh với các định dạng khác nhau
for filename in os.listdir(folder_path):
    # Check if the file has one of the allowed extensions
    if os.path.splitext(filename)[1].lower() in allowed_extensions:
        # Extract the base name without the extension to match with label_array
        basename = os.path.splitext(filename)[0]
        # Find the corresponding label for the current image file
        for i in label_array:
            if basename == str(i[0]):
                image_path = os.path.join(folder_path, filename)
                image = load_and_preprocess_image(image_path)
                images.append(image)
                Y_train.append(i[1])
                break  # Break the inner loop once the label is found





In [18]:
images = np.array(images)
Y_train = np.array(Y_train)

In [19]:
train_images, val_images, train_labels, val_labels = train_test_split(images, Y_train, test_size=0.2, random_state=42)

In [20]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout, BatchNormalization
from tensorflow.keras import regularizers
from tensorflow.keras.applications import VGG16

In [35]:
model = Sequential()

In [37]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(256, 256, 3))

In [39]:
model.add(base_model)
model.add(Flatten())
model.add(Dense(512, activation='relu', kernel_regularizer=regularizers.l2(0.001)))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))
base_model.trainable = False

In [41]:
model.compile('adam', loss=tf.losses.BinaryCrossentropy(), metrics=['accuracy'])

In [43]:
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ vgg16 (Functional)              │ ?                      │    14,714,688 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,714,688 (56.13 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 14,714,688 (56.13 MB)

In [52]:
history = model.fit(
    train_images, train_labels,
    epochs= 2,
    batch_size=32,
    validation_data=(val_images, val_labels)
)

NameError: name 'model' is not defined

In [67]:
loss, accuracy = model.evaluate(val_images, val_labels)
print(f'Validation loss: {loss}')
print(f'Validation accuracy: {accuracy}')

33/33 ━━━━━━━━━━━━━━━━━━━━ 132s 4s/step - accuracy: 0.8686 - loss: 0.4134
Validation loss: 0.412806898355484
Validation accuracy: 0.8617424368858337


In [ ]:
# Đường dẫn tới thư mục chứa ảnh
folder_path = "C:/Users/minhd/FPTU lab/DPL302m/Kaggle/2024-sum-dpl-302-m/testset_images/testset_images"

# Đọc và xử lý ảnh, sau đó dự đoán với mô hình
def predict_image(model, img_path, target_size=(256, 256)):
    image = Image.open(img_path)
    image = image.resize(target_size)
    image = np.array(image)
    if image.shape[-1] == 4:  # Nếu ảnh có kênh alpha (RGBA), chuyển về RGB
        image = image[:, :, :3]
    image = image / 255.0
    image = np.expand_dims(image, axis=0)  # Thêm chiều batch vào hình ảnh
    prediction = model.predict(image)
    return 1 if prediction[0][0] >= 0.5 else 0

test_label = []
id_test = []

# Danh sách các phần mở rộng file hợp lệ
allowed_extensions = ['.jpg', '.png', '.gif']

# Giả sử label_array là một mảng các tên file và nhãn tương ứng
# Cập nhật lại danh sách các file ảnh với các định dạng khác nhau
for filename in os.listdir(folder_path):
    # Kiểm tra nếu file có phần mở rộng hợp lệ
    if os.path.splitext(filename)[1].lower() in allowed_extensions:
        # Lấy tên file không có phần mở rộng để khớp với label_array
        basename = os.path.splitext(filename)[0]
        id_test.append(basename)
        # Tìm nhãn tương ứng cho file ảnh hiện tại
        image_path = os.path.join(folder_path, filename)
        output = predict_image(model, image_path)
        test_label.append(output)

print(test_label)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 318ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 138ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 138ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 134ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 136ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 138ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 136ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 138ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 137ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 153ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 144ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 138ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 138ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 133ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 137ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 136ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 141ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 136ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 139ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 135ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 137ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 131ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 146ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 137ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 133ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 

In [72]:
submit = pd.DataFrame({'id': id_test, 'label': test_label})
submit.head()

,id,label
0,100167480,1
1,10021457945,0
2,10025866403,0
3,10035741293,0
4,10126331405,0


In [74]:
results_csv_path = 'submit.csv'
submit.to_csv(results_csv_path, index=False)